# Blue books for bulldozers

The goal of the notebook is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuaration.  The data is sourced from auction result postings and includes information on usage and equipment configurations.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

/home/amit/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
import numpy as np
from sklearn import metrics

In [4]:
PATH = 'data/'
!ls {PATH}

Train.csv  Valid.csv


In [5]:
df_raw = pd.read_csv(f'{PATH}Train.csv',low_memory=False, parse_dates=['saledate'])

In [6]:
def display_all(df):
    with pd.option_context('display.max_rows', 1000):
        with pd.option_context('display.max_columns', 1000):
            display(df)

In [7]:
display_all(df_raw.tail().transpose())

,401120,401121,401122,401123,401124
SalesID,6333336,6333337,6333338,6333341,6333342
SalePrice,10500,11000,11500,9000,7750
MachineID,1840702,1830472,1887659,1903570,1926965
ModelID,21439,21439,21439,21435,21435
datasource,149,149,149,149,149
auctioneerID,1,1,1,2,2
YearMade,2005,2005,2005,2005,2005
MachineHoursCurrentMeter,NaN,NaN,NaN,NaN,NaN
UsageBand,NaN,NaN,NaN,NaN,NaN
saledate,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-11-02 00:00:00,2011-10-25 00:00:00,2011-10-25 00:00:00


In [41]:
df_raw.describe()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saleYear,saleMonth,saleWeek,saleDay,saleDayofweek,saleDayofyear,saleElapsed
count,4.011250e+05,401125.000000,4.011250e+05,401125.000000,401125.000000,380989.000000,401125.000000,1.427650e+05,401125.000000,401125.000000,401125.000000,401125.000000,401125.000000,401125.000000,4.011250e+05
mean,1.919713e+06,10.103096,1.217903e+06,6889.702980,134.665810,6.556040,1899.156901,3.457955e+03,2004.095728,6.407035,26.179864,16.110402,2.600975,179.977581,1.091433e+09
std,9.090215e+05,0.693621,4.409920e+05,6221.777842,8.962237,16.976779,291.797469,2.759026e+04,5.754190,3.424575,14.788059,8.427322,1.405763,103.559930,1.816977e+08
min,1.139246e+06,8.465900,0.000000e+00,28.000000,121.000000,0.000000,1000.000000,0.000000e+00,1989.000000,1.000000,1.000000,1.000000,0.000000,2.000000,6.009984e+08
25%,1.418371e+06,9.581904,1.088697e+06,3259.000000,132.000000,1.000000,1985.000000,0.000000e+00,2000.000000,3.000000,13.000000,9.000000,2.000000,84.000000,9.708768e+08
50%,1.639422e+06,10.085809,1.279490e+06,4604.000000,132.000000,2.000000,1995.000000,0.000000e+00,2006.000000,6.000000,25.000000,16.000000,3.000000,168.000000,1.143072e+09
75%,2.242707e+06,10.596635,1.468067e+06,8724.000000,136.000000,4.000000,2000.000000,3.025000e+03,2009.000000,9.000000,39.000000,23.000000,3.000000,271.000000,1.237853e+09
max,6.333342e+06,11.863582,2.486330e+06,37198.000000,172.000000,99.000000,2013.000000,2.483300e+06,2011.000000,12.000000,53.000000,31.000000,6.000000,365.000000,1.325203e+09


In [8]:
df_raw.SalePrice = np.log(df_raw.SalePrice) #convert sale price to log scale because compeition evaluation metrics is RMSLE

In [9]:
fld=df_raw.saledate

In [10]:
add_datepart(df_raw,'saledate')
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [11]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


In [12]:
train_cats(df_raw)

In [13]:
df_raw.UsageBand.cat.categories

Index(['High', 'Low', 'Medium'], dtype='object')

In [14]:
df_raw.UsageBand.cat.set_categories(['High','Medium','Low'], ordered=True, inplace=True)

In [15]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

Backhoe_Mounting            0.803872
Blade_Extension             0.937129
Blade_Type                  0.800977
Blade_Width                 0.937129
Coupler                     0.466620
Coupler_System              0.891660
Differential_Type           0.826959
Drive_System                0.739829
Enclosure                   0.000810
Enclosure_Type              0.937129
Engine_Horsepower           0.937129
Forks                       0.521154
Grouser_Tracks              0.891899
Grouser_Type                0.752813
Hydraulics                  0.200823
Hydraulics_Flow             0.891899
MachineHoursCurrentMeter    0.644089
MachineID                   0.000000
ModelID                     0.000000
Pad_Type                    0.802720
Pattern_Changer             0.752651
ProductGroup                0.000000
ProductGroupDesc            0.000000
ProductSize                 0.525460
Pushblock                   0.937129
Ride_Control                0.629527
Ripper                      0.740388
S

In [18]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/raw') #save data frame

In [19]:
df_raw = pd.read_feather('tmp/raw') #start here

In [20]:
df, y, nas = proc_df(df_raw, 'SalePrice')

In [21]:
proc_df

<function fastai.structured.proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None, preproc_fn=None, max_n_cat=None, subset=None, mapper=None)>

In [22]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

0.9881541589114123

In [23]:
#return copies of the array that can be modified without affecting the original array
def split_vals(a,n): return a[:n].copy(), a[n:].copy() 

n_valid = 12000 #same as Kaggle's test set size
n_trn = len(df) - n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)


X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((389125, 66), (389125,), (12000, 66), (12000,))

# Random Forest

In [24]:
def rmse(x,y):
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
          m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [25]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train) # %time shows the time it takes to complete a task
print_score(m)

CPU times: user 15min 13s, sys: 2.05 s, total: 15min 15s
Wall time: 1min 59s
[0.07580188012085007, 0.23509093419392463, 0.9879913335726189, 0.9012994125546683]


validation score 0.90 vs training score 0.98 shows that it is over fitting

# Speeding things up

Using only a subset of the training data while keeping the same amount of validcation data as before for accuracy

In [28]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset = 30000, na_dict = nas)
X_train, _ = split_vals(df_trn, 20000) # _ is throw-away variable
y_train, _ = split_vals(y_trn, 20000)

In [29]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 29.4 s, sys: 24.5 ms, total: 29.4 s
Wall time: 3.99 s
[0.09367084705884349, 0.3633582890250942, 0.9810382468075535, 0.7642139441650002]


# Bagging

Out-of-bag score: allow us to see whether our model generalizes, even if we only have a small amount of data so to avoid separating some out to create a validation set Use SubSampling to avoid overfitting while increase speed.

In [32]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

In [33]:
set_rf_samples(20000) #Instead of limiting the total amount of data accessible, use different random subset per tree

In [34]:
m = RandomForestRegressor(n_jobs=-1, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 15min 23s, sys: 2.54 s, total: 15min 26s
Wall time: 2min 6s
[0.07561297730927669, 0.23425890814807226, 0.9880511116220647, 0.9019968119588786, 0.9129267589583753]


In [35]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.07848251024892229, 0.23778830452889807, 0.9871269754664308, 0.8990214908216662, 0.9083397692005049]


# Tree building parameters

grow trees less deep

In [37]:
reset_rf_samples()
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.11509254937503784, 0.23378793761967684, 0.9723159741405908, 0.9023904807877438, 0.9085198291640226]


using different sets of features(columns) for each split in a tree

In [39]:
m = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, n_jobs=-1, max_features=0.5, oob_score=True) #use random 50% of the features on each split
m.fit(X_train, y_train)
print_score(m)

[0.11746075403066676, 0.2262361165903815, 0.9711649708382601, 0.9085946009994461, 0.915335672042507]


In [40]:
m = RandomForestRegressor(n_estimators=100, min_samples_leaf=3, n_jobs=-1, max_features='sqrt', oob_score=True) #try sqrt
m.fit(X_train, y_train)
print_score(m)

[0.15578398034595864, 0.2574507538212359, 0.9492798629715815, 0.8816314549132914, 0.8998328671299352]
